## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
VOCAB_PATH = BERT_PATH + '/assets/vocab.txt'
model_path_list = glob.glob('/kaggle/input/132-googleq-a-train-3fold-bert-base-unc-q-a3-1e5/' + '*.h5')
model_path_list += glob.glob('/kaggle/input/133-googleq-a-train-3fold-bert-base-unc-q-a3-5e5/' + '*.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/132-googleq-a-train-3fold-bert-base-unc-q-a3-1e5/model_fold_1.h5
/kaggle/input/132-googleq-a-train-3fold-bert-base-unc-q-a3-1e5/model_fold_2.h5
/kaggle/input/132-googleq-a-train-3fold-bert-base-unc-q-a3-1e5/model_fold_3.h5
/kaggle/input/133-googleq-a-train-3fold-bert-base-unc-q-a3-5e5/model_fold_1.h5
/kaggle/input/133-googleq-a-train-3fold-bert-base-unc-q-a3-5e5/model_fold_2.h5
/kaggle/input/133-googleq-a-train-3fold-bert-base-unc-q-a3-5e5/model_fold_3.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
text_features_question = ['question_title', 'question_body']
text_features_answer = ['question_title', 'answer']

# for feature in text_features:
#     # Lower
#     test[feature] = test[feature].apply(lambda x: x.lower())
#     # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
#     # Map contractions
#     test[feature] = test[feature].apply(lambda x: map_contraction(x))
#     # Trim text
#     test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
N_CLASS_QUESTION = len(question_target_cols)
N_CLASS_ANSWER = len(answer_target_cols)
MAX_SEQUENCE_LENGTH = 512
MAX_LENGTH_TITLE = 30
MAX_LENGTH_QUESTION = 479
MAX_LENGTH_ANSWER = 479

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays_2(test, text_features_question, tokenizer, MAX_SEQUENCE_LENGTH, 
                               MAX_LENGTH_TITLE, MAX_LENGTH_QUESTION, ignore_first_sep=False)\
        + compute_input_arays_2(test, text_features_answer, tokenizer, MAX_SEQUENCE_LENGTH, 
                                MAX_LENGTH_TITLE, MAX_LENGTH_ANSWER, ignore_first_sep=False)

# Model

In [7]:
def model_fn():
    input_word_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_question')
    input_masks_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_question')
    segment_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_question')

    input_word_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_answer')
    input_masks_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_answer')
    segment_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_answer')

    bert_layer = KerasLayer(BERT_PATH, trainable=True)
    pooled_output_question, sequence_output_question = bert_layer([input_word_ids_question, input_masks_question, segment_ids_question])
    pooled_output_answer, sequence_output_answer = bert_layer([input_word_ids_answer, input_masks_answer, segment_ids_answer])

    x_question = GlobalAveragePooling1D()(sequence_output_question)
    x_question = Dropout(0.2)(x_question)
    output_question = Dense(N_CLASS_QUESTION, kernel_initializer='glorot_uniform', activation="sigmoid", name="outputquestion")(x_question)
    
    x_answer = GlobalAveragePooling1D()(sequence_output_answer)
    x_answer = Dropout(0.2)(x_answer)
    output_answer = Dense(N_CLASS_ANSWER, kernel_initializer='glorot_uniform', activation="sigmoid", name="output_answer")(x_answer)
    
    output = Concatenate()([output_question, output_answer])

    model = Model(inputs=[input_word_ids_question, input_masks_question, segment_ids_question, 
                          input_word_ids_answer, input_masks_answer, segment_ids_answer], outputs=output)

    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.943722,0.691237,0.209314,0.538197,0.545862,0.550426,0.686648,0.603966,0.586498,...,0.889627,0.920271,0.631430,0.958740,0.966505,0.838160,0.024834,0.035971,0.744238,0.906918
1,46,0.853324,0.531371,0.010060,0.713400,0.754675,0.907136,0.534448,0.430529,0.111207,...,0.586635,0.939123,0.648176,0.967060,0.977356,0.870900,0.900576,0.152498,0.123390,0.875528
2,70,0.886405,0.691325,0.030929,0.728531,0.861323,0.894081,0.588486,0.477768,0.140032,...,0.842629,0.943726,0.597497,0.965013,0.966025,0.853826,0.034671,0.038333,0.829815,0.899975
3,132,0.843749,0.424097,0.009935,0.684140,0.737498,0.895220,0.538250,0.415211,0.136058,...,0.636518,0.941140,0.687409,0.970212,0.979272,0.893278,0.750855,0.163618,0.620457,0.879419
4,200,0.934748,0.503044,0.057005,0.812619,0.739954,0.856234,0.653774,0.577624,0.127738,...,0.659327,0.904261,0.652275,0.955668,0.956884,0.823168,0.211481,0.131535,0.582014,0.880473


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.889130,0.603901,0.041150,0.687490,0.774012,0.828531,0.582218,0.471082,0.236637,...,0.760688,0.929545,0.665828,0.960410,0.970520,0.867418,0.507383,0.140388,0.509575,0.896260
std,2812.670060,0.048306,0.140248,0.060988,0.097711,0.098672,0.121562,0.051851,0.079620,0.192678,...,0.100600,0.027619,0.045279,0.013270,0.013057,0.038380,0.314825,0.061354,0.269879,0.024737
min,39.000000,0.741639,0.327984,0.005533,0.262232,0.395165,0.248727,0.490129,0.335947,0.019975,...,0.556140,0.750278,0.504065,0.894315,0.883054,0.721160,0.009144,0.013098,0.048416,0.818656
25%,2572.000000,0.853646,0.475520,0.010688,0.632079,0.737844,0.802363,0.540769,0.414004,0.091316,...,0.671964,0.917598,0.636535,0.954732,0.964465,0.847359,0.186871,0.096930,0.275785,0.880633
50%,5093.000000,0.892750,0.583667,0.016031,0.685116,0.781356,0.875093,0.571398,0.443016,0.162506,...,0.761287,0.936705,0.662896,0.962578,0.973752,0.875777,0.589116,0.147681,0.503099,0.898732
75%,7482.000000,0.927908,0.726556,0.038500,0.742786,0.832666,0.906080,0.620223,0.511347,0.356212,...,0.856250,0.948499,0.691967,0.969290,0.979513,0.896400,0.801754,0.184957,0.747821,0.914722
max,9640.000000,0.976108,0.897602,0.487411,0.952196,0.968402,0.956520,0.729643,0.717510,0.825773,...,0.944980,0.973662,0.808802,0.981763,0.988901,0.941377,0.936699,0.325060,0.968156,0.949859
